In [1]:
import tensorflow as tf
import numpy as np
import gc
import pandas as pd 
from tensorflow.keras.layers import ConvLSTM2D, Conv3D, Conv2D, Flatten, Dense, BatchNormalization
import tensorflow.math as M
from utils import *
from sklearn.model_selection import train_test_split
from video_loader import DeepFakeDualTransformer, DeepFakeTransformer
import sys
import pathlib
import datetime
import matplotlib.pyplot as plt

In [2]:

class SimilarityLoss(tf.keras.losses.Loss):
    # Loss function subclassed to penalize for returning a pair 
    # with a high cosine sim
    def call(self, y_real, y_fake):
        cosine_sim = M.reduce_sum(M.multiply(y_real, y_fake))/(M.reduce_euclidean_norm(y_real) * M.reduce_euclidean_norm(y_fake))
        loss = M.scalar_mul(-1, M.log(1 - cosine_sim))

        return loss
@tf.function
def similarity_loss(y_real, y_fake):
    
    cosine_sim = M.reduce_sum(M.multiply(y_real, y_fake))/(M.reduce_euclidean_norm(y_real) * M.reduce_euclidean_norm(y_fake))
    loss = M.scalar_mul(-1, M.log(1 - cosine_sim))
    
    return loss

def load_from_file_pair(real_path, fake_path, transformer):
  
    real_vid = transformer.transform_vid(real_path)[0]
    fake_vid = transformer.transform_vid(fake_path)[0]

    return tf.stack((real_vid, fake_vid))

In [3]:
# Define some params
# model params
load_ckpt = False # set false if not loading


# Train params
EPOCHS=10
batch_size=1
reg_penalty = 0.001
cls_wt = {0:3, 1:2.25}
validate_epochs = list(np.arange(EPOCHS))


# Dataset params
data_pairs_path = '../data/source/labels/fake_to_real_mapping.csv'
resize_shape = (224,224)
sequence_len = 30
prefetch_num = 4
train_val_split = 0.015


# Final model params
dt = datetime.datetime.now()
tstamp = f'{dt.year}{dt.month}{dt.day}{dt.hour}:{dt.minute}:{dt.second}'
regstr = str(reg_penalty).split('.')[1]

model_stamp = tstamp + f'_model_{regstr}_reg'
final_model_path = f'models/{model_stamp}/model.h5'
checkpoint_prefix = f'models/{model_stamp}/'

In [4]:
# Losses, metrics, optimizer

# loss = SimilarityLoss

# Define dummy test dims based on parameters
test_dims = (None, None, *resize_shape, 3)

# Get device names

cpu = tf.config.experimental.list_physical_devices('CPU')[0].name
print("FOUND CPU AT: ", cpu)
print([x[0] for x in tf.config.experimental.list_physical_devices('GPU')])


# Define and load the datasets
df_pairs = pd.read_csv(data_pairs_path)[['real', 'fake']]
train_df, val_df = train_test_split(df_pairs, test_size = train_val_split)


FOUND CPU AT:  /physical_device:CPU:0
['/physical_device:GPU:0']


In [5]:
# # Create dataset from pairs of path strings
# train_ds = tf.data.Dataset.from_tensor_slices(train_df.to_numpy())
# val_ds = tf.data.Dataset.from_tensor_slices(val_df.to_numpy())

# # TODO add in random crops, rotations, etc to make this non-redundant
# # define the transformer to load data on the fly
# train_transformer = DeepFakeDualTransformer(resize_shape=resize_shape, seq_length=sequence_len)
# val_transformer = DeepFakeDualTransformer(resize_shape=resize_shape, seq_length=sequence_len)

# # map the transformer on the dataset entries
# train_ds = train_ds.map(lambda x: train_transformer.transform_map(x)).prefetch(prefetch_num)
# val_ds = val_ds.map(lambda x: val_transformer.transform_map(x)).prefetch(prefetch_num)


X_train_len = len(list(train_df))
X_val_len = len(list(val_df))
num_train_batches = int(np.ceil(X_train_len/batch_size))
num_val_batches = int(np.ceil(X_val_len/batch_size))

transformer = DeepFakeTransformer(resize_shape=resize_shape, seq_length=sequence_len) 

lr_fn = tf.keras.optimizers.schedules.PolynomialDecay(0.001, num_train_batches*EPOCHS, 
                                                      end_learning_rate=1e-5)
optimizer = tf.keras.optimizers.SGD(lr_fn)

In [6]:
# Define the model, currently copying model archs to model.py manually
reg = tf.keras.regularizers.l2(l=reg_penalty)
model = tf.keras.models.Sequential()
model.add(ConvLSTM2D(64, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last',
                            recurrent_regularizer=reg,
                            kernel_regularizer=reg,
                            bias_regularizer=reg, 
                            return_sequences=True,
                            input_shape=test_dims[1:]))
model.add(ConvLSTM2D(128, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last',
                            recurrent_regularizer=reg,
                            kernel_regularizer=reg,
                            bias_regularizer=reg,  
                            return_sequences=True,
                            ))
model.add(ConvLSTM2D(128, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last',
                            recurrent_regularizer=reg,
                            kernel_regularizer=reg,
                            bias_regularizer=reg, 
                            return_sequences=True,
                            ))
model.add(ConvLSTM2D(128, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last',
                            recurrent_regularizer=reg,
                            kernel_regularizer=reg,
                            bias_regularizer=reg,  
                            return_sequences=False,
                            ))
model.add(Conv2D(64 , (3,3), strides=(2,2), 
                        padding='valid', data_format='channels_last',
                        kernel_regularizer=reg,
                        bias_regularizer=reg,
                        activation='relu'))
model.add(Conv2D(64 , (3,3), strides=(2,2), 
                        padding='valid', data_format='channels_last',
                        kernel_regularizer=reg,
                        bias_regularizer=reg,
                        activation='relu'))

model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.compile()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, None, 111, 111, 64 154624    
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 55, 55, 128) 885248    
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 27, 27, 128) 1180160   
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 13, 13, 128)       1180160   
_________________________________________________________________
conv2d (Conv2D)              (None, 6, 6, 64)          73792     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

In [7]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, checkpoint_prefix, max_to_keep=5)

# Keep results for plotting
train_loss_results = []
validation_loss_results = []

if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
else:
    print("Initializing from scratch.")


Initializing from scratch.


In [8]:

@tf.function
def grad(model, x, loss):
    with tf.GradientTape() as tape:
        y_pair = model(x)
        loss_value = loss(y_pair[0], y_pair[1])
    grads = tape.gradient(loss_value, model.trainable_variables)
    return loss_value, grads

In [9]:
for epoch in range(EPOCHS):
        epoch_loss_avg = tf.keras.metrics.Mean()

        i = 0
        # Training loop
        for rp, fp in train_df.to_numpy():
            
            vids = load_from_file_pair(rp, fp, transformer)

            # Optimize the model
            loss_value, grads = grad(model, vids, similarity_loss)
            
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            print("Training batch {} loss: {:.3f}".format(i, loss_value))
            # Track progress
            epoch_loss_avg(loss_value)  # Add current batch loss
            i += 1

        # End epoch ops
        train_loss_results.append(epoch_loss_avg.result())
        manager.save(checkpoint_number=epoch)

        
        print("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))

Training batch 0 loss: 15.249
Training batch 1 loss: 13.170
Training batch 2 loss: 14.844
Training batch 3 loss: 15.942
Training batch 4 loss: 14.844
Training batch 5 loss: 12.785
Training batch 6 loss: 13.417
Training batch 7 loss: 15.942
Training batch 8 loss: 15.942
Training batch 9 loss: 14.333
Training batch 10 loss: 12.610
Training batch 11 loss: 15.942
Training batch 12 loss: 14.844
Training batch 13 loss: 14.333
Training batch 14 loss: 12.922
Training batch 15 loss: 13.996
Training batch 16 loss: 14.844
Training batch 17 loss: 13.996
Training batch 18 loss: 15.249
Training batch 19 loss: 11.891
Training batch 20 loss: 14.556
Training batch 21 loss: 15.249
Training batch 22 loss: 14.556
Training batch 23 loss: 13.109
Training batch 24 loss: 15.942
Training batch 25 loss: 14.556
Training batch 26 loss: 14.556
Training batch 27 loss: inf
Training batch 28 loss: nan
Training batch 29 loss: nan
Training batch 30 loss: nan
Training batch 31 loss: nan
Training batch 32 loss: nan
Train

TypeError: 'NoneType' object is not iterable

In [ ]:
model(np.random.randn(2,30,224,224,3))